# Used Car Pricing Analysis
This analysis will provide prediction of used car prices by evaluating which factors are important to buyers and how they impact the valuation.



## Data
Dataset of this analysis is from Kaggle of 426k car sales data

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder, OrdinalEncoder

In [2]:
sales = pd.read_csv('data/vehicles.csv')

In [3]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

In [4]:
sales.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
0,7222695916,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az
1,7218891961,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar
2,7221797935,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl
3,7222270760,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma
4,7210384030,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc


In [5]:
sales.tail()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
426875,7301591192,wyoming,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,gas,32226.0,clean,other,1N4AA6AV6KC367801,fwd,NaN,sedan,NaN,wy
426876,7301591187,wyoming,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,gas,12029.0,clean,other,7JR102FKXLG042696,fwd,NaN,sedan,red,wy
426877,7301591147,wyoming,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,diesel,4174.0,clean,other,1GYFZFR46LF088296,NaN,NaN,hatchback,white,wy
426878,7301591140,wyoming,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,gas,30112.0,clean,other,58ABK1GG4JU103853,fwd,NaN,sedan,silver,wy
426879,7301591129,wyoming,30590,2019.0,bmw,4 series 430i gran coupe,good,NaN,gas,22716.0,clean,other,WBA4J1C58KBM14708,rwd,NaN,coupe,NaN,wy


### Data cleaning

* Drop unnecessary columns
* Remove records without essential data
* Transform data

In [6]:
# Drop id and VIN
sales = sales.drop(['id', 'VIN'], axis = 1)
sales

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az
1,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar
2,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl
3,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma
4,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,wyoming,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,gas,32226.0,clean,other,fwd,NaN,sedan,NaN,wy
426876,wyoming,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,gas,12029.0,clean,other,fwd,NaN,sedan,red,wy
426877,wyoming,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,diesel,4174.0,clean,other,NaN,NaN,hatchback,white,wy
426878,wyoming,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,gas,30112.0,clean,other,fwd,NaN,sedan,silver,wy


In [7]:
# Unique values in each column
for col in sales.columns:
    print(col)
    print(sales[col].unique())

region
['prescott' 'fayetteville' 'florida keys' 'worcester / central MA'
 'greensboro' 'hudson valley' 'medford-ashland' 'erie' 'el paso'
 'bellingham' 'skagit / island / SJI' 'la crosse' 'auburn' 'birmingham'
 'dothan' 'florence / muscle shoals' 'gadsden-anniston'
 'huntsville / decatur' 'mobile' 'montgomery' 'tuscaloosa'
 'anchorage / mat-su' 'fairbanks' 'kenai peninsula' 'southeast alaska'
 'flagstaff / sedona' 'mohave county' 'phoenix' 'show low' 'sierra vista'
 'tucson' 'yuma' 'fort smith' 'jonesboro' 'little rock' 'texarkana'
 'bakersfield' 'chico' 'fresno / madera' 'gold country' 'hanford-corcoran'
 'humboldt county' 'imperial county' 'inland empire' 'los angeles'
 'mendocino county' 'merced' 'modesto' 'monterey bay' 'orange county'
 'palm springs' 'redding' 'reno / tahoe' 'sacramento' 'san diego'
 'san luis obispo' 'santa barbara' 'santa maria' 'SF bay area'
 'siskiyou county' 'stockton' 'susanville' 'ventura county'
 'visalia-tulare' 'yuba-sutter' 'boulder' 'colorado springs'

In [8]:
# Too many unique values in region, state, manufacturer and model to keep forecasting model simple. 
# Paint color is irrelevant
sales = sales.drop(['region', 'state', 'manufacturer', 'model', 'paint_color'], axis = 1)

Remove records with missing data

In [9]:
# Remove rows with no data in price, year and odometer
sales = sales.dropna(subset=['price', 'year', 'odometer', 'condition', 'cylinders', 'fuel'])


Keep only 'clean' title status since used car dealers will unlikely take on other types of title

In [10]:
sales = sales[sales['title_status']=='clean']
sales = sales.drop('title_status', axis=1)

## Remove records with bad data or outliers for rare cars
* Price or odometer with 0 or large number like 12345678 are bad records
* Cars unlikely to be accepted by used car dealership
  * Priced lower than `$1000`
  * Mileage more than `200k`
  * Older than `20` years are 
* Price greater than $50k are rare or custom cars that would not be effective to include in model
* Mileage too low are new cars that would not be useful for forecast

In [11]:
# Toss out price that are too low (less than $1000) or too high (more than $50,000)
sales = sales[(sales['price'] >= 1000) & (sales['price'] <= 50000) ]

In [12]:
# Toss out cars with odometer that are too low or too high 
sales = sales[(sales['odometer'] >= 1000) & (sales['odometer'] <= 200000) ]

In [14]:
# Toss out cars older than 25 years
sales = sales[(sales['year'] > 2023 - 25) ]

In [15]:
sales.describe()

,price,year,odometer
count,129364.000000,129364.000000,129364.000000
mean,17370.789277,2011.770029,95766.107286
std,12109.415471,5.114280,52776.080633
min,1000.000000,1999.000000,1000.000000
25%,6997.000000,2008.000000,49734.500000
50%,13900.000000,2013.000000,98000.000000
75%,26990.000000,2016.000000,137679.250000
max,50000.000000,2022.000000,200000.000000


### Transform values

Change year of manufacture to age_in_years

In [16]:
sales['year'].max(skipna=True)

2022.0

Assume data is from 2023, since 2022 is the max year value

In [17]:
sales['years_old'] = 2023 - sales['year']
sales = sales.drop('year', axis=1)

Remove records with invalid data in 'cylinders' column

In [19]:
# remove other and empty rows
sales = sales[sales['cylinders']!='other']

# Convert the 'cylinders' column to integers
sales['cylinders'] = sales['cylinders'].str.replace(' cylinders', '')
sales['cylinders'] = sales['cylinders'].astype(int)


In [20]:
# Set missing value to 'other'

sales.fillna({
    'transmission':'other',
    'drive':'other',
    'size':'other',
    'type':'other'
}, inplace=True)

In [21]:
# Check Unique values in each column
for col in sales.columns:
    print(col)
    print(sales[col].unique())

price
[33590 22590 39590 ... 10914 27294 19853]
condition
[3. 2. 4. 0. 1. 5.]
cylinders
[ 8  6  4  5  3 10 12]
fuel
['gas' 'diesel' 'other' 'hybrid' 'electric']
odometer
[ 57923.  71229.  19160. ... 130626. 176261. 172511.]
transmission
['other' 'automatic' 'manual']
drive
['other' 'rwd' '4wd' 'fwd']
size
['other' 'full-size' 'mid-size' 'compact' 'sub-compact']
type
['pickup' 'truck' 'other' 'coupe' 'mini-van' 'SUV' 'sedan' 'hatchback'
 'bus' 'convertible' 'wagon' 'van' 'offroad']
years_old
[ 9. 13.  3.  6. 10. 11.  7.  4. 12.  5. 19.  8. 22. 17. 20. 16. 18. 15.
 14. 21. 24. 23.  2.  1.]


In [22]:
# Check for empty f
sales.isna().sum()

price           0
condition       0
cylinders       0
fuel            0
odometer        0
transmission    0
drive           0
size            0
type            0
years_old       0
dtype: int64

In [25]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 129025 entries, 27 to 426878
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         129025 non-null  int64  
 1   condition     129025 non-null  float64
 2   cylinders     129025 non-null  int64  
 3   fuel          129025 non-null  object 
 4   odometer      129025 non-null  float64
 5   transmission  129025 non-null  object 
 6   drive         129025 non-null  object 
 7   size          129025 non-null  object 
 8   type          129025 non-null  object 
 9   years_old     129025 non-null  float64
dtypes: float64(3), int64(2), object(5)
memory usage: 10.8+ MB


In [24]:
sales.describe()

,price,condition,cylinders,odometer,years_old
count,129025.000000,129025.000000,129025.000000,129025.000000,129025.000000
mean,17378.913156,2.370300,5.921930,95831.130231,11.228878
std,12109.429141,0.702641,1.545755,52757.318513,5.112881
min,1000.000000,0.000000,3.000000,1000.000000,1.000000
25%,6998.000000,2.000000,4.000000,49798.000000,7.000000
50%,13900.000000,2.000000,6.000000,98000.000000,10.000000
75%,26990.000000,3.000000,8.000000,137776.000000,15.000000
max,50000.000000,5.000000,12.000000,200000.000000,24.000000


#### Data cleaning completed
* No missing values
* Non-categorical data range and mean looks reasonable
* Good number of records remaining to perform analysis

Transform ordinal columns:
* condition

In [18]:
oe = OrdinalEncoder(categories = [['new', 'like new', 'excellent', 'good', 'fair', 'salvage']])
sales['condition'] = oe.fit_transform(sales[['condition']])